In [533]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, accuracy_score, recall_score


In [534]:
data_sucio = pd.read_csv('DataAnalytics.csv')
data_sucio.head(5)

,Administrador,Usuario,botón correcto,tiempo de interacción,mini juego,número de interacción,color presionado,dificultad,fecha,Juego,auto push,tiempo de lección,tiempo de sesión
0,nicolas,nicolas,1.0,5.399169,Despegue,1.0,blue,Episodio 1,25/01/2024 09:26:42 a. m.,Astro,0.0,0.0,0.0
1,nicolas,nicolas,0.0,1.283400,Despegue,2.0,violet,Episodio 1,25/01/2024 09:26:46 a. m.,Astro,0.0,0.0,0.0
2,nicolas,nicolas,1.0,2.700226,Despegue,3.0,green,Episodio 1,25/01/2024 09:26:48 a. m.,Astro,0.0,0.0,0.0
3,nicolas,nicolas,0.0,3.050262,Despegue,4.0,green,Episodio 1,25/01/2024 09:26:57 a. m.,Astro,0.0,0.0,0.0
4,nicolas,nicolas,0.0,4.750256,Despegue,5.0,green,Episodio 1,25/01/2024 09:26:58 a. m.,Astro,0.0,0.0,0.0


In [535]:
data_sucio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5865 entries, 0 to 5864
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Administrador          5865 non-null   object 
 1   Usuario                5865 non-null   object 
 2   botón correcto         5103 non-null   float64
 3   tiempo de interacción  5103 non-null   float64
 4   mini juego             5709 non-null   object 
 5   número de interacción  5103 non-null   float64
 6   color presionado       5103 non-null   object 
 7   dificultad             5865 non-null   object 
 8   fecha                  5865 non-null   object 
 9   Juego                  5865 non-null   object 
 10  auto push              5103 non-null   float64
 11  tiempo de lección      5688 non-null   float64
 12  tiempo de sesión       5259 non-null   float64
dtypes: float64(6), object(7)
memory usage: 595.8+ KB


## Limpieza de Nulos 

In [536]:
data_cuanti=data_sucio.select_dtypes(include=["float64","int64","float","int"])
data_cuali=data_sucio.select_dtypes(include=["object","datetime","category","datetime64[ns]"])
data_cuali.info()
data_cuanti.info(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5865 entries, 0 to 5864
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Administrador     5865 non-null   object
 1   Usuario           5865 non-null   object
 2   mini juego        5709 non-null   object
 3   color presionado  5103 non-null   object
 4   dificultad        5865 non-null   object
 5   fecha             5865 non-null   object
 6   Juego             5865 non-null   object
dtypes: object(7)
memory usage: 320.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5865 entries, 0 to 5864
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   botón correcto         5103 non-null   float64
 1   tiempo de interacción  5103 non-null   float64
 2   número de interacción  5103 non-null   float64
 3   auto push              5103 non-null   float64
 4   tiempo de lecci

In [537]:
cualis_bfill=data_cuali.fillna(method="bfill")
cualis_ffill=data_cuali.fillna(method="ffill")
cuantias_mean=data_cuanti.fillna(round(data_cuanti.mean(),1))

data_sin_nulos = pd.concat([cuantias_mean,cualis_ffill],axis=1)

data_sin_nulos.head(5)

C:\Users\B_3mi\AppData\Local\Temp\ipykernel_16060\3586926086.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cualis_bfill=data_cuali.fillna(method="bfill")
C:\Users\B_3mi\AppData\Local\Temp\ipykernel_16060\3586926086.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cualis_ffill=data_cuali.fillna(method="ffill")


,botón correcto,tiempo de interacción,número de interacción,auto push,tiempo de lección,tiempo de sesión,Administrador,Usuario,mini juego,color presionado,dificultad,fecha,Juego
0,1.0,5.399169,1.0,0.0,0.0,0.0,nicolas,nicolas,Despegue,blue,Episodio 1,25/01/2024 09:26:42 a. m.,Astro
1,0.0,1.283400,2.0,0.0,0.0,0.0,nicolas,nicolas,Despegue,violet,Episodio 1,25/01/2024 09:26:46 a. m.,Astro
2,1.0,2.700226,3.0,0.0,0.0,0.0,nicolas,nicolas,Despegue,green,Episodio 1,25/01/2024 09:26:48 a. m.,Astro
3,0.0,3.050262,4.0,0.0,0.0,0.0,nicolas,nicolas,Despegue,green,Episodio 1,25/01/2024 09:26:57 a. m.,Astro
4,0.0,4.750256,5.0,0.0,0.0,0.0,nicolas,nicolas,Despegue,green,Episodio 1,25/01/2024 09:26:58 a. m.,Astro


In [538]:
data=data_sin_nulos.isnull().sum().sum()
print(data)

0


In [539]:
DATA = data

## Limpieza Outliers

In [540]:
cuantitativas=data_sucio.select_dtypes(include=["float64","int64","float","int"])
cualitativas=data_sucio.select_dtypes(include=["object","datetime","category"])
y=cuantitativas

percentile25=y.quantile(0.25)
percentile75=y.quantile(0.75)
iqr= percentile75-percentile25

Limite_Superior_iqr= percentile75+1.5*iqr
Limite_Inferior_iqr= percentile25-1.5*iqr
iqr=cuantitativas[(y<=Limite_Superior_iqr)&y>=(Limite_Inferior_iqr)]
iqr2=iqr.fillna(round(iqr.mean(),1))
rango_intercuartilico=pd.concat([cualitativas,iqr2],axis=1)
rango_intercuartilico.head(5)

,Administrador,Usuario,mini juego,color presionado,dificultad,fecha,Juego,botón correcto,tiempo de interacción,número de interacción,auto push,tiempo de lección,tiempo de sesión
0,nicolas,nicolas,Despegue,blue,Episodio 1,25/01/2024 09:26:42 a. m.,Astro,1.0,5.399169,1.0,0.0,0.0,0.0
1,nicolas,nicolas,Despegue,violet,Episodio 1,25/01/2024 09:26:46 a. m.,Astro,0.0,1.283400,2.0,0.0,0.0,0.0
2,nicolas,nicolas,Despegue,green,Episodio 1,25/01/2024 09:26:48 a. m.,Astro,1.0,2.700226,3.0,0.0,0.0,0.0
3,nicolas,nicolas,Despegue,green,Episodio 1,25/01/2024 09:26:57 a. m.,Astro,0.0,3.050262,4.0,0.0,0.0,0.0
4,nicolas,nicolas,Despegue,green,Episodio 1,25/01/2024 09:26:58 a. m.,Astro,0.0,4.750256,5.0,0.0,0.0,0.0


In [541]:
cualis_bfill=data_cuali.fillna(method="bfill")
cualis_ffill=data_cuali.fillna(method="ffill")
cuantias_mean=data_cuanti.fillna(round(data_cuanti.mean(),1))

data = pd.concat([cuantias_mean,cualis_ffill],axis=1)

data.head(5)

C:\Users\B_3mi\AppData\Local\Temp\ipykernel_16060\3308866133.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cualis_bfill=data_cuali.fillna(method="bfill")
C:\Users\B_3mi\AppData\Local\Temp\ipykernel_16060\3308866133.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cualis_ffill=data_cuali.fillna(method="ffill")


,botón correcto,tiempo de interacción,número de interacción,auto push,tiempo de lección,tiempo de sesión,Administrador,Usuario,mini juego,color presionado,dificultad,fecha,Juego
0,1.0,5.399169,1.0,0.0,0.0,0.0,nicolas,nicolas,Despegue,blue,Episodio 1,25/01/2024 09:26:42 a. m.,Astro
1,0.0,1.283400,2.0,0.0,0.0,0.0,nicolas,nicolas,Despegue,violet,Episodio 1,25/01/2024 09:26:46 a. m.,Astro
2,1.0,2.700226,3.0,0.0,0.0,0.0,nicolas,nicolas,Despegue,green,Episodio 1,25/01/2024 09:26:48 a. m.,Astro
3,0.0,3.050262,4.0,0.0,0.0,0.0,nicolas,nicolas,Despegue,green,Episodio 1,25/01/2024 09:26:57 a. m.,Astro
4,0.0,4.750256,5.0,0.0,0.0,0.0,nicolas,nicolas,Despegue,green,Episodio 1,25/01/2024 09:26:58 a. m.,Astro


In [542]:
outliers_limpios=data.isnull().sum().sum()
print(outliers_limpios)

0


## Analizar la correlación logística (botón correcto, juego, auto push)

In [543]:
df = pd.read_csv('DataAnalytics.csv')
df.head()

,Administrador,Usuario,botón correcto,tiempo de interacción,mini juego,número de interacción,color presionado,dificultad,fecha,Juego,auto push,tiempo de lección,tiempo de sesión
0,nicolas,nicolas,1.0,5.399169,Despegue,1.0,blue,Episodio 1,25/01/2024 09:26:42 a. m.,Astro,0.0,0.0,0.0
1,nicolas,nicolas,0.0,1.283400,Despegue,2.0,violet,Episodio 1,25/01/2024 09:26:46 a. m.,Astro,0.0,0.0,0.0
2,nicolas,nicolas,1.0,2.700226,Despegue,3.0,green,Episodio 1,25/01/2024 09:26:48 a. m.,Astro,0.0,0.0,0.0
3,nicolas,nicolas,0.0,3.050262,Despegue,4.0,green,Episodio 1,25/01/2024 09:26:57 a. m.,Astro,0.0,0.0,0.0
4,nicolas,nicolas,0.0,4.750256,Despegue,5.0,green,Episodio 1,25/01/2024 09:26:58 a. m.,Astro,0.0,0.0,0.0


In [544]:
# Crear un diccionario para almacenar los modelos de regresión logística
logistic_models = {}

In [545]:
# Preparamos los datos, eliminando nulos y codificando 'Juego'
df_clean = df[['botón correcto', 'auto push', 'Juego', 'número de interacción', 'tiempo de interacción']].dropna()

In [546]:
# Convertir 'Juego' en variable binaria para regresión logística (0 si es el juego más común, 1 si no)
juego_mas_comun = df_clean['Juego'].mode()[0]
df_clean['Juego_binaria'] = (df_clean['Juego'] != juego_mas_comun).astype(int)

In [547]:
df_clean['Juego_binaria']

0       0
1       0
2       0
3       0
4       0
5       0
6       0
7       0
8       0
9       0
10      0
11      0
13      0
14      0
15      0
16      0
17      0
18      0
19      0
21      0
22      0
23      0
24      0
25      0
26      0
28      0
29      0
30      0
31      0
32      0
33      0
34      0
35      0
36      0
37      0
38      0
39      0
41      0
42      0
43      0
44      0
45      0
46      0
47      0
48      0
49      0
51      0
52      0
53      0
54      0
55      0
56      0
57      0
58      0
59      0
60      0
61      0
62      0
64      0
65      0
66      0
67      0
68      0
69      0
70      0
71      0
72      0
73      0
74      0
75      0
76      0
77      0
78      0
79      0
81      0
82      0
83      0
84      0
85      0
86      0
87      0
88      0
89      0
90      0
91      0
92      0
93      0
94      0
95      0
96      0
97      0
98      0
99      0
100     0
101     0
102     0
103     0
104     0
105     0
106     0


In [548]:
df_clean.head()

,botón correcto,auto push,Juego,número de interacción,tiempo de interacción,Juego_binaria
0,1.0,0.0,Astro,1.0,5.399169,0
1,0.0,0.0,Astro,2.0,1.283400,0
2,1.0,0.0,Astro,3.0,2.700226,0
3,0.0,0.0,Astro,4.0,3.050262,0
4,0.0,0.0,Astro,5.0,4.750256,0


In [549]:
Var_Indepx= df_clean[['número de interacción', 'tiempo de interacción']]
Var_Depy1=df_clean['botón correcto']
Var_Depy2=df_clean['auto push']
Var_Depy3=df_clean['Juego_binaria']

In [550]:
#Redefinimos las variables 
X= Var_Indepx
y1= Var_Depy1
y2= Var_Depy2
y3= Var_Depy3

In [551]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y1, test_size=0.3, random_state =None)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y1, test_size=0.3, random_state =None)
X_train3, X_test3, y_train3, y_test3 = train_test_split(X, y1, test_size=0.3, random_state =None)

In [552]:
#Se escalan todos los datos
escalar = StandardScaler()

In [553]:
#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train1 = escalar.fit_transform(X_train1)
X_test1 = escalar.transform(X_test1)

X_train2 = escalar.fit_transform(X_train2)
X_test2 = escalar.transform(X_test2)

X_train3 = escalar.fit_transform(X_train3)
X_test3 = escalar.transform(X_test3)

In [554]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [555]:
#Entrenamos el modelo
algoritmo.fit(X_train1, y_train1)
algoritmo.fit(X_train2, y_train2)
algoritmo.fit(X_train3, y_train3)

LogisticRegression()

In [556]:
#Realizamos una predicción
y_pred1 = algoritmo.predict(X_test1) 
y_pred1

y_pred2 = algoritmo.predict(X_test2) 
y_pred2

y_pred3 = algoritmo.predict(X_test3) 
y_pred3

array([1., 1., 1., ..., 0., 1., 0.], shape=(1531,))

In [557]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz1 = confusion_matrix(y_test1, y_pred1)
print('Matriz de Confusión 1: ')
print(matriz1)

matriz2 = confusion_matrix(y_test2, y_pred2)
print('Matriz de Confusión 2: ')
print(matriz2)

matriz3 = confusion_matrix(y_test3, y_pred3)
print('Matriz de Confusión 3: ')
print(matriz3)

Matriz de Confusión 1: 
[[370 418]
 [114 629]]
Matriz de Confusión 2: 
[[356 401]
 [124 650]]
Matriz de Confusión 3: 
[[378 417]
 [126 610]]


In [558]:
# Modelo 1: botón correcto ~ número de interacción + tiempo de interacción
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

print("Modelo 1: Botón Correcto 0.0")
precision1_0= precision_score (y_test1, y_pred1, average="binary", pos_label=0.0)
print('Precisión del modelo: ',precision1_0 )
exactitud1_0 = accuracy_score(y_test1, y_pred1)
print('Exactitud del modelo: ', exactitud1_0)
sensibilidad1_0 = recall_score(y_test1, y_pred1, average="binary", pos_label=0.0)
print('Sensibilidad del modelo: ', sensibilidad1_0)
puntajef1_1_0 = f1_score(y_test1, y_pred1, average="binary", pos_label=0.0)
print('Puntaje F1 del modelo: ', puntajef1_1_0)
print("\n")

print("Modelo 1: Botón correcto 1.0 ")
precision1= precision_score (y_test1, y_pred1, average="binary", pos_label=1.0)
print('Precisión del modelo: ',precision1 )
exactitud1 = accuracy_score(y_test1, y_pred1)
print('Exactitud del modelo: ', exactitud1)
sensibilidad1 = recall_score(y_test1, y_pred1, average="binary", pos_label=1.0)
print('Sensibilidad del modelo: ', sensibilidad1)
puntajef1_1 = f1_score(y_test1, y_pred1, average="binary", pos_label=1.0)
print('Puntaje F1 del modelo: ', puntajef1_1)

Modelo 1: Botón Correcto 0.0
Precisión del modelo:  0.7644628099173554
Exactitud del modelo:  0.6525146962769431
Sensibilidad del modelo:  0.46954314720812185
Puntaje F1 del modelo:  0.5817610062893082


Modelo 1: Botón correcto 1.0 
Precisión del modelo:  0.600764087870105
Exactitud del modelo:  0.6525146962769431
Sensibilidad del modelo:  0.8465679676985195
Puntaje F1 del modelo:  0.7027932960893855


In [559]:
# Modelo 2: Auto Push ~ número de interacción + tiempo de interacción
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

print("Modelo 2: Auto push 0.0")
precision2_0= precision_score (y_test2, y_pred2, average="binary", pos_label=0.0)
print('Precisión del modelo: ',precision2_0 )
exactitud2_0 = accuracy_score(y_test2, y_pred2)
print('Exactitud del modelo: ', exactitud2_0)
sensibilidad2_0 = recall_score(y_test2, y_pred2, average="binary", pos_label=0.0)
print('Sensibilidad del modelo: ', sensibilidad2_0)
puntajef1_2_0 = f1_score(y_test2, y_pred2, average="binary", pos_label=0.0)
print('Puntaje F1 del modelo: ', puntajef1_2_0)
print("\n")


print("Modelo 2: Auto push 1.0")
precision2= precision_score (y_test2, y_pred2, average="binary", pos_label=1.0)
print('Precisión del modelo: ',precision2 )
exactitud2 = accuracy_score(y_test2, y_pred2)
print('Exactitud del modelo: ', exactitud2)
sensibilidad2 = recall_score(y_test2, y_pred2, average="binary", pos_label=1.0)
print('Sensibilidad del modelo: ', sensibilidad2)
puntajef1_2 = f1_score(y_test2, y_pred2, average="binary", pos_label=1.0)
print('Puntaje F1 del modelo: ', puntajef1_2)

Modelo 2: Auto push 0.0
Precisión del modelo:  0.7416666666666667
Exactitud del modelo:  0.6570868713259308
Sensibilidad del modelo:  0.4702774108322325
Puntaje F1 del modelo:  0.5755860953920776


Modelo 2: Auto push 1.0
Precisión del modelo:  0.6184586108468125
Exactitud del modelo:  0.6570868713259308
Sensibilidad del modelo:  0.8397932816537468
Puntaje F1 del modelo:  0.7123287671232876


In [560]:
# Modelo 3: Juego ~ número de interacción + tiempo de interacción
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

print("Modelo 3: Juego 0.0")
precision3_0= precision_score (y_test3, y_pred3, average="binary", pos_label=0.0)
print('Precisión del modelo: ',precision3_0 )
exactitud3_0 = accuracy_score(y_test3, y_pred3)
print('Exactitud del modelo: ', exactitud3_0)
sensibilidad3_0 = recall_score(y_test3, y_pred3, average="binary", pos_label=0.0)
print('Sensibilidad del modelo: ', sensibilidad3_0)
puntajef1_3_0 = f1_score(y_test1, y_pred1, average="binary", pos_label=0.0)
print('Puntaje F1 del modelo: ', puntajef1_3_0)
print("\n")

print("Modelo 3: Juego 1.0")
precision3= precision_score (y_test3, y_pred3, average="binary", pos_label=1.0)
print('Precisión del modelo: ',precision3 )
exactitud3 = accuracy_score(y_test3, y_pred3)
print('Exactitud del modelo: ', exactitud3)
sensibilidad3 = recall_score(y_test3, y_pred3, average="binary", pos_label=1.0)
print('Sensibilidad del modelo: ', sensibilidad3)
puntajef1_3 = f1_score(y_test1, y_pred1, average="binary", pos_label=1.0)
print('Puntaje F1 del modelo: ', puntajef1_3)

Modelo 3: Juego 0.0
Precisión del modelo:  0.75
Exactitud del modelo:  0.6453298497713913
Sensibilidad del modelo:  0.47547169811320755
Puntaje F1 del modelo:  0.5817610062893082


Modelo 3: Juego 1.0
Precisión del modelo:  0.5939629990262901
Exactitud del modelo:  0.6453298497713913
Sensibilidad del modelo:  0.8288043478260869
Puntaje F1 del modelo:  0.7027932960893855


## Convertir las variables en tipo dicotómica y analizarlas

Función de Regresión Logística

In [561]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

def regresioncion_logistica(dicotomica,si,no):
    dicotomica=dicotomica.replace(si,"0")
    dicotomica=dicotomica.replace(no,"1")
    dicotomica = dicotomica.apply(pd.to_numeric, errors="coerce")
    Var_Indepx= data[['número de interacción', 'tiempo de interacción']]
    Var_Depy=dicotomica
    X= Var_Indepx
    Y= Var_Depy
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state =None)
    escalar = StandardScaler()
    X_train = escalar.fit_transform(X_train)
    X_test = escalar.transform(X_test)
    algoritmo = LogisticRegression()
    algoritmo.fit(X_train, Y_train)
    Y_pred = algoritmo.predict(X_test) 
    matriz = confusion_matrix(Y_test, Y_pred)
    print('Matriz de Confusión: ')
    print(matriz)
    print("Modelo 0:")
    precision0= precision_score (Y_test, Y_pred, average="binary", pos_label=0.0)
    print('Precisión del modelo: ',precision0 )
    exactitud0 = accuracy_score(Y_test, Y_pred)
    print('Exactitud del modelo: ', exactitud0)
    sensibilidad0 = recall_score(Y_test, Y_pred, average="binary", pos_label=0.0)
    print('Sensibilidad del modelo: ', sensibilidad0)
    puntajef0 = f1_score(Y_test, Y_pred, average="binary", pos_label=0.0)
    print('Puntaje F1 del modelo: ', puntajef0)
    print("\n")
    print("Modelo 1:")
    precision1= precision_score (Y_test, Y_pred, average="binary", pos_label=1.0)
    print('Precisión del modelo: ',precision1 )
    exactitud1 = accuracy_score(Y_test, Y_pred)
    print('Exactitud del modelo: ', exactitud1)
    sensibilidad1 = recall_score(Y_test, Y_pred, average="binary", pos_label=1.0)
    print('Sensibilidad del modelo: ', sensibilidad1)
    puntajef1_1 = f1_score(Y_test, Y_pred, average="binary", pos_label=1.0)
    print('Puntaje F1 del modelo: ', puntajef1_1)

Color Presionado (5)

In [562]:
copr = np.unique(data_sin_nulos['color presionado'])
copr

array(['blue', 'green', 'red', 'violet', 'yellow'], dtype=object)

In [563]:
blue= data_sin_nulos['color presionado'].replace(["green", "red", "violet", "yellow"], "no blue")
blue.head()

0       blue
1    no blue
2    no blue
3    no blue
4    no blue
Name: color presionado, dtype: object

In [564]:
blue_relo=regresioncion_logistica(blue,"blue","no blue")

Matriz de Confusión: 
[[   0  401]
 [   0 1359]]
Modelo 0:
Precisión del modelo:  0.0
Exactitud del modelo:  0.772159090909091
Sensibilidad del modelo:  0.0
Puntaje F1 del modelo:  0.0


Modelo 1:
Precisión del modelo:  0.772159090909091
Exactitud del modelo:  0.772159090909091
Sensibilidad del modelo:  1.0
Puntaje F1 del modelo:  0.8714331516511703


c:\Users\B_3mi\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [565]:
green= data_sin_nulos['color presionado'].replace(["blue", "red", "violet", "yellow"], "no green")
green.head()

0    no green
1    no green
2       green
3       green
4       green
Name: color presionado, dtype: object

In [566]:
green_relo=regresioncion_logistica(green,"green","no green")

Matriz de Confusión: 
[[   0  462]
 [   0 1298]]
Modelo 0:
Precisión del modelo:  0.0
Exactitud del modelo:  0.7375
Sensibilidad del modelo:  0.0
Puntaje F1 del modelo:  0.0


Modelo 1:
Precisión del modelo:  0.7375
Exactitud del modelo:  0.7375
Sensibilidad del modelo:  1.0
Puntaje F1 del modelo:  0.8489208633093526


c:\Users\B_3mi\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [567]:
red= data_sin_nulos['color presionado'].replace(["blue", "green", "violet", "yellow"], "no red")
red.head()

0    no red
1    no red
2    no red
3    no red
4    no red
Name: color presionado, dtype: object

In [568]:
red_relo=regresioncion_logistica(red,"red","no red")

Matriz de Confusión: 
[[   0   14]
 [   0 1746]]
Modelo 0:


c:\Users\B_3mi\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Precisión del modelo:  0.0
Exactitud del modelo:  0.9920454545454546
Sensibilidad del modelo:  0.0
Puntaje F1 del modelo:  0.0


Modelo 1:
Precisión del modelo:  0.9920454545454546
Exactitud del modelo:  0.9920454545454546
Sensibilidad del modelo:  1.0
Puntaje F1 del modelo:  0.9960068454078722


In [569]:
violet= data_sin_nulos['color presionado'].replace(["blue", "green", "red", "yellow"], "no violet")
violet.head()

0    no violet
1       violet
2    no violet
3    no violet
4    no violet
Name: color presionado, dtype: object

In [570]:
violet_relo=regresioncion_logistica(violet,"violet","no violet")

Matriz de Confusión: 
[[  59  408]
 [   3 1290]]
Modelo 0:
Precisión del modelo:  0.9516129032258065
Exactitud del modelo:  0.7664772727272727
Sensibilidad del modelo:  0.12633832976445397
Puntaje F1 del modelo:  0.22306238185255198


Modelo 1:
Precisión del modelo:  0.7597173144876325
Exactitud del modelo:  0.7664772727272727
Sensibilidad del modelo:  0.9976798143851509
Puntaje F1 del modelo:  0.8625877632898696


In [571]:
yellow= data_sin_nulos['color presionado'].replace(["blue", "green", "red", "violet"], "no yellow")
yellow.head()

0    no yellow
1    no yellow
2    no yellow
3    no yellow
4    no yellow
Name: color presionado, dtype: object

In [572]:
yellow_relo=regresioncion_logistica(yellow,"yellow","no yellow")

Matriz de Confusión: 
[[   0  413]
 [   0 1347]]
Modelo 0:
Precisión del modelo:  0.0
Exactitud del modelo:  0.7653409090909091
Sensibilidad del modelo:  0.0
Puntaje F1 del modelo:  0.0


Modelo 1:
Precisión del modelo:  0.7653409090909091
Exactitud del modelo:  0.7653409090909091
Sensibilidad del modelo:  1.0
Puntaje F1 del modelo:  0.8670743482458964


c:\Users\B_3mi\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Dificultad

In [573]:
dif = np.unique(data_sin_nulos['dificultad'])
dif

array(['Episodio 1', 'Episodio 2', 'Episodio 3', 'Episodio 4'],
      dtype=object)

In [574]:
E1= data_sin_nulos['dificultad'].replace(["Episodio 2", "Episodio 3", "Episodio 4"], "NO Episodio 1")
E1.head()

0    Episodio 1
1    Episodio 1
2    Episodio 1
3    Episodio 1
4    Episodio 1
Name: dificultad, dtype: object

In [575]:
E1_relo=regresioncion_logistica(E1,"Episodio 1","NO Episodio 1")

Matriz de Confusión: 
[[ 105  604]
 [  46 1005]]
Modelo 0:
Precisión del modelo:  0.695364238410596
Exactitud del modelo:  0.6306818181818182
Sensibilidad del modelo:  0.14809590973201692
Puntaje F1 del modelo:  0.2441860465116279


Modelo 1:
Precisión del modelo:  0.6246115599751398
Exactitud del modelo:  0.6306818181818182
Sensibilidad del modelo:  0.956232159847764
Puntaje F1 del modelo:  0.7556390977443609


In [576]:
E2= data_sin_nulos['dificultad'].replace(["Episodio 1", "Episodio 3", "Episodio 4"], "NO Episodio 2")
E2.head()

0    NO Episodio 2
1    NO Episodio 2
2    NO Episodio 2
3    NO Episodio 2
4    NO Episodio 2
Name: dificultad, dtype: object

In [577]:
E2_relo=regresioncion_logistica(E2,"Episodio 2","NO Episodio 2")

Matriz de Confusión: 
[[   0  444]
 [   0 1316]]
Modelo 0:
Precisión del modelo:  0.0
Exactitud del modelo:  0.7477272727272727
Sensibilidad del modelo:  0.0
Puntaje F1 del modelo:  0.0


Modelo 1:
Precisión del modelo:  0.7477272727272727
Exactitud del modelo:  0.7477272727272727
Sensibilidad del modelo:  1.0
Puntaje F1 del modelo:  0.8556566970091027


c:\Users\B_3mi\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [578]:
E3= data_sin_nulos['dificultad'].replace(["Episodio 1", "Episodio 2", "Episodio 4"], "NO Episodio 3")
E3.head()

0    NO Episodio 3
1    NO Episodio 3
2    NO Episodio 3
3    NO Episodio 3
4    NO Episodio 3
Name: dificultad, dtype: object

In [579]:
E3_relo=regresioncion_logistica(E3,"Episodio 3","NO Episodio 3")

Matriz de Confusión: 
[[   1  376]
 [   1 1382]]
Modelo 0:
Precisión del modelo:  0.5
Exactitud del modelo:  0.7857954545454545
Sensibilidad del modelo:  0.002652519893899204
Puntaje F1 del modelo:  0.005277044854881266


Modelo 1:
Precisión del modelo:  0.7861205915813424
Exactitud del modelo:  0.7857954545454545
Sensibilidad del modelo:  0.9992769342010123
Puntaje F1 del modelo:  0.8799745304043298


In [580]:
E4= data_sin_nulos['dificultad'].replace(["Episodio 1", "Episodio 2", "Episodio 3"], "NO Episodio 4")
E4.head()

0    NO Episodio 4
1    NO Episodio 4
2    NO Episodio 4
3    NO Episodio 4
4    NO Episodio 4
Name: dificultad, dtype: object

In [581]:
E4_relo=regresioncion_logistica(E4,"Episodio 4","NO Episodio 4")

Matriz de Confusión: 
[[   0  233]
 [   0 1527]]
Modelo 0:
Precisión del modelo:  0.0
Exactitud del modelo:  0.8676136363636363
Sensibilidad del modelo:  0.0
Puntaje F1 del modelo:  0.0


Modelo 1:
Precisión del modelo:  0.8676136363636363
Exactitud del modelo:  0.8676136363636363
Sensibilidad del modelo:  1.0
Puntaje F1 del modelo:  0.9291146942500761


c:\Users\B_3mi\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Mini Juego(2)

In [582]:
#numero de interacción, usuario(5))
miju = np.unique(data_sin_nulos['mini juego'])
miju

array(['Asteroides', 'Despegue', 'Estrellas', 'Gusanos', 'MiniGame_0',
       'MiniGame_1', 'MiniGame_2', 'MiniGame_3', 'Restaurante',
       'animales y colores', 'figuras y colores', 'partes del cuerpo',
       'sonidos y animales'], dtype=object)

In [583]:
Asteroides= data_sin_nulos['mini juego'].replace(["Despegue", "Estrellas", "Gusanos", "MiniGame_0", "MiniGame_1", "MiniGame_2", "MiniGame_3", "Restaurante", "animales y colores", "figuras y colores", "partes del cuerpo", "sonidos y animales"], "NO Asteroides")
Asteroides.head()

0    NO Asteroides
1    NO Asteroides
2    NO Asteroides
3    NO Asteroides
4    NO Asteroides
Name: mini juego, dtype: object

In [584]:
Asteroides_relo=regresioncion_logistica(Asteroides,"Asteroides","NO Asteroides")

Matriz de Confusión: 
[[   0  317]
 [   1 1442]]
Modelo 0:
Precisión del modelo:  0.0
Exactitud del modelo:  0.8193181818181818
Sensibilidad del modelo:  0.0
Puntaje F1 del modelo:  0.0


Modelo 1:
Precisión del modelo:  0.8197839681637293
Exactitud del modelo:  0.8193181818181818
Sensibilidad del modelo:  0.9993069993069993
Puntaje F1 del modelo:  0.9006870705808869


In [585]:
Despegue= data_sin_nulos['mini juego'].replace(["Asteroides", "Estrellas", "Gusanos", "MiniGame_0", "MiniGame_1", "MiniGame_2", "MiniGame_3", "Restaurante", "animales y colores", "figuras y colores", "partes del cuerpo", "sonidos y animales"], "NO Despegue")
Despegue.head()

0    Despegue
1    Despegue
2    Despegue
3    Despegue
4    Despegue
Name: mini juego, dtype: object

In [586]:
Despegue_relo=regresioncion_logistica(Despegue,"Despegue","NO Despegue")

Matriz de Confusión: 
[[   0   62]
 [   0 1698]]
Modelo 0:
Precisión del modelo:  0.0
Exactitud del modelo:  0.9647727272727272
Sensibilidad del modelo:  0.0
Puntaje F1 del modelo:  0.0


Modelo 1:
Precisión del modelo:  0.9647727272727272
Exactitud del modelo:  0.9647727272727272
Sensibilidad del modelo:  1.0
Puntaje F1 del modelo:  0.9820705610179294


c:\Users\B_3mi\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Numero de Interacción

In [587]:
Max=data_sin_nulos['número de interacción'].max()
Min=data_sin_nulos['número de interacción'].min()
Limites= [Min, Max]
Limites

[np.float64(1.0), np.float64(256.0)]

In [588]:
intervalos=np.linspace(0.9, 256.1, 3)
print(intervalos)
categorias= ["Interacción Baja", "Interacción Alta"]

[  0.9 128.5 256.1]


In [589]:
pd.options.display.max_rows = None

In [590]:
internew=pd.cut(x= data_sin_nulos['número de interacción'], bins=intervalos, labels= categorias )
internew.head()

0    Interacción Baja
1    Interacción Baja
2    Interacción Baja
3    Interacción Baja
4    Interacción Baja
Name: número de interacción, dtype: category
Categories (2, object): ['Interacción Baja' < 'Interacción Alta']

In [591]:
internew_relo=regresioncion_logistica(internew,"Interacción Alta","Interacción Baja")

Matriz de Confusión: 
[[  35    0]
 [   1 1724]]
Modelo 0:
Precisión del modelo:  0.9722222222222222
Exactitud del modelo:  0.9994318181818181
Sensibilidad del modelo:  1.0


C:\Users\B_3mi\AppData\Local\Temp\ipykernel_16060\521905368.py:9: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  dicotomica=dicotomica.replace(si,"0")
C:\Users\B_3mi\AppData\Local\Temp\ipykernel_16060\521905368.py:10: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  dicotomica=dicotomica.replace(no,"1")


Puntaje F1 del modelo:  0.9859154929577465


Modelo 1:
Precisión del modelo:  1.0
Exactitud del modelo:  0.9994318181818181
Sensibilidad del modelo:  0.9994202898550725
Puntaje F1 del modelo:  0.9997100608872137


Usuario (5)

In [592]:
us = np.unique(data_sin_nulos['Usuario'])
us

array(['ADRIAN ', 'ALEIDA', 'ARLETT', 'ASHLEY ', 'AUSTIN', 'BENJAMIN',
       'CARLOS ABEL', 'CARLOS ENRIQUE', 'CONCEPCION', 'DENISSE',
       'ERICK OSVALDO', 'ESMERALDA', 'IKER BENJAMIN', 'INGRID', 'IRVING ',
       'JESUS ALEJANDRO', 'JOSE IAN', 'JOSE IGNACIO TADEO',
       'JOSE JAVIER ', 'JOSHUA', 'KYTZIA', 'LEONARDO', 'LEONARDO ',
       'MA DEL ROSARIO ', 'NICOLAS |', 'RENE ', 'SERGIO ANGEL',
       'VALENTIN', 'YAEL DAVID', 'YEREMI YAZMIN ', 'erick ',
       'jesus eduardo', 'nicolas', 'ramiro isai'], dtype=object)

In [593]:
ADRIAN= data_sin_nulos['Usuario'].replace(['ALEIDA', 'ARLETT', 'ASHLEY ', 'AUSTIN', 'BENJAMIN','CARLOS ABEL', 'CARLOS ENRIQUE', 'CONCEPCION', 'DENISSE','ERICK OSVALDO', 'ESMERALDA', 'IKER BENJAMIN', 'INGRID', 'IRVING ','JESUS ALEJANDRO', 'JOSE IAN', 'JOSE IGNACIO TADEO','JOSE JAVIER ', 'JOSHUA', 'KYTZIA', 'LEONARDO', 'LEONARDO ','MA DEL ROSARIO ', 'NICOLAS |', 'RENE ', 'SERGIO ANGEL','VALENTIN', 'YAEL DAVID', 'YEREMI YAZMIN ', 'erick ','jesus eduardo', 'nicolas', 'ramiro isai'], "NO ADRIAN")
ADRIAN.head()

0    NO ADRIAN
1    NO ADRIAN
2    NO ADRIAN
3    NO ADRIAN
4    NO ADRIAN
Name: Usuario, dtype: object

In [ ]:
ADRIAN_relo=regresioncion_logistica(ADRIAN,"ADRIAN","NO ADRIAN")

In [ ]:
ALEIDA= data_sin_nulos['Usuario'].replace(['ADRIAN', 'ARLETT', 'ASHLEY ', 'AUSTIN', 'BENJAMIN','CARLOS ABEL', 'CARLOS ENRIQUE', 'CONCEPCION', 'DENISSE','ERICK OSVALDO', 'ESMERALDA', 'IKER BENJAMIN', 'INGRID', 'IRVING ','JESUS ALEJANDRO', 'JOSE IAN', 'JOSE IGNACIO TADEO','JOSE JAVIER ', 'JOSHUA', 'KYTZIA', 'LEONARDO', 'LEONARDO ','MA DEL ROSARIO ', 'NICOLAS |', 'RENE ', 'SERGIO ANGEL','VALENTIN', 'YAEL DAVID', 'YEREMI YAZMIN ', 'erick ','jesus eduardo', 'nicolas', 'ramiro isai'], "NO ALEIDA")
ALEIDA.head()

0    NO ALEIDA
1    NO ALEIDA
2    NO ALEIDA
3    NO ALEIDA
4    NO ALEIDA
Name: Usuario, dtype: object

In [ ]:
ALEIDA_relo=regresioncion_logistica(ALEIDA,"ALEIDA","NO ALEIDA")

In [ ]:
ARLETT= data_sin_nulos['Usuario'].replace(['ADRIAN', 'ALEIDA', 'ASHLEY ', 'AUSTIN', 'BENJAMIN','CARLOS ABEL', 'CARLOS ENRIQUE', 'CONCEPCION', 'DENISSE','ERICK OSVALDO', 'ESMERALDA', 'IKER BENJAMIN', 'INGRID', 'IRVING ','JESUS ALEJANDRO', 'JOSE IAN', 'JOSE IGNACIO TADEO','JOSE JAVIER ', 'JOSHUA', 'KYTZIA', 'LEONARDO', 'LEONARDO ','MA DEL ROSARIO ', 'NICOLAS |', 'RENE ', 'SERGIO ANGEL','VALENTIN', 'YAEL DAVID', 'YEREMI YAZMIN ', 'erick ','jesus eduardo', 'nicolas', 'ramiro isai'], "NO ARLETT")
ARLETT.head()

0    NO ARLETT
1    NO ARLETT
2    NO ARLETT
3    NO ARLETT
4    NO ARLETT
Name: Usuario, dtype: object

In [ ]:
ARLETT_relo=regresioncion_logistica(ARLETT,"ARLETT","NO ARLETT")

In [ ]:
ASHLEY= data_sin_nulos['Usuario'].replace(['ADRIAN', 'ALEIDA', 'ARLETT', 'AUSTIN', 'BENJAMIN','CARLOS ABEL', 'CARLOS ENRIQUE', 'CONCEPCION', 'DENISSE','ERICK OSVALDO', 'ESMERALDA', 'IKER BENJAMIN', 'INGRID', 'IRVING ','JESUS ALEJANDRO', 'JOSE IAN', 'JOSE IGNACIO TADEO','JOSE JAVIER ', 'JOSHUA', 'KYTZIA', 'LEONARDO', 'LEONARDO ','MA DEL ROSARIO ', 'NICOLAS |', 'RENE ', 'SERGIO ANGEL','VALENTIN', 'YAEL DAVID', 'YEREMI YAZMIN ', 'erick ','jesus eduardo', 'nicolas', 'ramiro isai'], "NO ASHLEY")
ASHLEY.head()

0    NO ASHLEY
1    NO ASHLEY
2    NO ASHLEY
3    NO ASHLEY
4    NO ASHLEY
Name: Usuario, dtype: object

In [ ]:
ASHLEY_relo=regresioncion_logistica(ASHLEY,"ASHLEY","NO ASHLEY")

In [ ]:
AUSTIN= data_sin_nulos['Usuario'].replace(['ADRIAN', 'ALEIDA', 'ARLETT', 'ASHLEY ', 'BENJAMIN','CARLOS ABEL', 'CARLOS ENRIQUE', 'CONCEPCION', 'DENISSE','ERICK OSVALDO', 'ESMERALDA', 'IKER BENJAMIN', 'INGRID', 'IRVING ','JESUS ALEJANDRO', 'JOSE IAN', 'JOSE IGNACIO TADEO','JOSE JAVIER ', 'JOSHUA', 'KYTZIA', 'LEONARDO', 'LEONARDO ','MA DEL ROSARIO ', 'NICOLAS |', 'RENE ', 'SERGIO ANGEL','VALENTIN', 'YAEL DAVID', 'YEREMI YAZMIN ', 'erick ','jesus eduardo', 'nicolas', 'ramiro isai'], "NO AUSTIN")
AUSTIN.head()

0    NO AUSTIN
1    NO AUSTIN
2    NO AUSTIN
3    NO AUSTIN
4    NO AUSTIN
Name: Usuario, dtype: object

In [ ]:
AUSTIN_relo=regresioncion_logistica(AUSTIN,"AUSTIN","NO AUSTIN")

## Realizar una tabla de todos los coeficientes de precisión, exactitud y sensibilidad 

Botón correcto

In [ ]:
tabla_metricas = pd.DataFrame({
    'Clase': ['Botón correcto 0.0', 'Botón correcto 1.0'],
    'Precisión': [precision1_0, precision1],
    'Exactitud': [exactitud1_0, exactitud1],
    'Sensibilidad (Recall)': [sensibilidad1_0, sensibilidad1],
    'Puntaje F1': [puntajef1_1_0, puntajef1_1]
})

# Mostrar tabla
print("Métricas de Botón Correcto: \n", tabla_metricas)

Métricas de Botón Correcto: 
                 Clase  Precisión  Exactitud  Sensibilidad (Recall)  Puntaje F1
0  Botón correcto 0.0   0.742000   0.660353               0.486877    0.587956
1  Botón correcto 1.0   0.620757   0.660353               0.832250    0.711111


Juego

In [ ]:
tabla_metricas = pd.DataFrame({
    'Clase': ['Juego 0.0', 'Juego 1.0'],
    'Precisión': [precision2_0, precision2],
    'Exactitud': [exactitud2_0, exactitud2],
    'Sensibilidad (Recall)': [sensibilidad2_0, sensibilidad2],
    'Puntaje F1': [puntajef1_2_0, puntajef1_2]
})

# Mostrar tabla
print("Métricas de Juego: \n",tabla_metricas)

Métricas de Juego: 
        Clase  Precisión  Exactitud  Sensibilidad (Recall)  Puntaje F1
0  Juego 0.0   0.748454   0.648596               0.465982    0.574367
1  Juego 1.0   0.602294   0.648596               0.837766    0.700779


Auto push

In [ ]:
tabla_metricas = pd.DataFrame({
    'Clase': ['Auto Push 0.0', 'Auto Push 1.0'],
    'Precisión': [precision3_0, precision3],
    'Exactitud': [exactitud3_0, exactitud3],
    'Sensibilidad (Recall)': [sensibilidad3_0, sensibilidad3],
    'Puntaje F1': [puntajef1_3_0, puntajef1_3]
})

# Mostrar tabla
print("Métricas de Auto Push: \n",tabla_metricas)

Métricas de Auto Push: 
            Clase  Precisión  Exactitud  Sensibilidad (Recall)  Puntaje F1
0  Auto Push 0.0   0.771836   0.679295                0.54397    0.587956
1  Auto Push 1.0   0.625773   0.679295                0.82585    0.711111
